In [1]:
import torch
import torch.nn as nn

class SimpleRNNForTokenClassification(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_tags):
        super(SimpleRNNForTokenClassification, self).__init__()
        # 1. Lớp Embedding: Chuyển đổi ID của từ thành vector dày đặc (dense vector).
        # Ví dụ: từ có ID là 5 -> vector [0.1, 0.9, ..., 0.4]
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        # 2. Lớp RNN: Xử lý chuỗi vector và tạo ra hidden state tại mỗi bước.
        # batch_first=True nghĩa là input và output sẽ có chiều batch ở đầu tiên.
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        # 3. Lớp Linear: Ánh xạ từ hidden state (kích thước hidden_dim)
        # sang không gian số lượng nhãn (kích thước num_tags).
        self.linear = nn.Linear(hidden_dim, num_tags)

    def forward(self, sentence):
        # `sentence` là một chuỗi các ID của từ, có dạng (batch_size, seq_len).

        # 1. Truyền câu qua lớp Embedding.
        # Output `embeds` có dạng (batch_size, seq_len, embedding_dim).
        embeds = self.embedding(sentence)

        # 2. Truyền chuỗi embedding qua lớp RNN.
        # `rnn_out` là output của RNN tại mỗi bước thời gian.
        # `rnn_out` có dạng (batch_size, seq_len, hidden_dim).
        rnn_out, _ = self.rnn(embeds)

        # 3. Truyền output của RNN qua lớp Linear để lấy điểm số cho mỗi nhãn.
        # `tag_scores` có dạng (batch_size, seq_len, num_tags).
        tag_scores = self.linear(rnn_out)
        # (Trong thực tế, chúng ta sẽ áp dụng Softmax lên tag_scores để có phân phối xác suất)
        return tag_scores

In [4]:
vocab_size = 100
embedding_dim = 10
hidden_dim = 8
num_tags = 5

model = SimpleRNNForTokenClassification(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_dim=hidden_dim, num_tags=num_tags)

input_data = torch.LongTensor([[1,7,9,2,0,2]]) # một câu gồm 4 từ
output_data = model(input_data)

In [6]:
output_data.shape

torch.Size([1, 6, 5])